# 集思录转债

- 作者：DGC'Idea
- 版本：V0.1   
- 更新日期：2018年3月20日

----------

- 研究目的：获取可转债的相关信息；
- 研究方法：从集思录网站爬取可转债相关数据。

In [10]:
#导入库
import numpy as np
import pandas as pd

import json
import os

#引入自定义库
import sys
sys.path.append('../DS')
from ds_jisilu import *

#设置pd显示行列数目
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',100)
pd.set_option('display.width',10000)

#已上市可转债
def get_cb(data_type):
    df=get_jsl_data(data_type)
    #过滤掉可交换债
    df=df[df['btype']=='C']
    #更改列标题
    df=df.rename(columns={
        'bond_id':'代码','bond_nm':'转债名称','price':'现价','increase_rt':'涨跌',
        'stock_id':'正股代码','stock_nm':'正股名称','sprice':'股价','sincrease_rt':'涨跌 ',
        'convert_dt':'转股起始日','convert_price':'转股价','convert_value':'转股价值',
        'premium_rt':'溢价率','convert_amt_ratio':'转债比',
        'force_redeem_price':'强赎触发价','redeem_price_ratio':'强赎触发比',
        'put_price':'回售价',   
        'next_put_dt':'回售起始日','put_convert_price':'回售触发价','put_convert_price_ratio':'回售触发比',             
        'short_maturity_dt':'到期时间','year_left':'剩余(年)','redeem_price':'到期赎回价',    
        'ytm_rt':'税前收益','ytm_rt_tax':'税后收益',           
        'orig_iss_amt':'规模(亿)','curr_iss_amt':'剩余规模(亿)','volume':'成交额(万)','repo_discount_rt':'折算率',           
        'put_tc':'回售条款', 'redeem_tc':'强赎条款','cpn_desc':'利息','adjust_tc':'向下修正',                        
         }) 
    return df

#待发、待上市可转债
def get_cbnew(data_type):
    df=get_jsl_data(data_type)
    #过滤掉可交换债
    df=df[df['cb_type']=='可转债']
    #更改列标题
    df=df.rename(columns={
        'bond_id':'代码','bond_nm':'转债名称',
        'stock_id':'正股代码','stock_nm':'正股名称',
        'amount':'发行规模(亿)',
        'progress_nm':'进展','progress_dt':'进展日期',
        'cb_amount':'百元股票含权(元)',
        'convert_price':'转股价','price':'现价','increase_rt':'涨幅','pma_rt':'现价比转股价',
        'ration_rt':'股东配售率','online_amount':'网上规模(亿)',
        'lucky_draw_rt':'中签率','single_draw':'单账户中签(顶格)','valid_apply':'申购户数(万)',   
         }) 
    #整理进展信息   
    for i in range(len(df)):
        #判断是否是已发
        if df['cp_flag'].iloc[i]=='Y':
            #清除多余字符
            progress=df['进展'].iloc[i].replace('<br>','')
            df['进展日期'].iloc[i]=progress[0:10]
            if progress[10:12]=='上市':
                df['进展'].iloc[i]=progress[10:12]
            else:
                df['进展'].iloc[i]=progress[10:12]+'（'+progress[14:]+'）'
    return df

try:
    cbnew_df=get_cbnew('new_cbond')
    cb_df=get_cb('sell_cbond')
    print(' 转债数据已准备好！')
except Exception as e:
    print(' 转债数据获取失败！')


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
 转债数据获取失败！


## 1.已上市转债

### 1.1.基本信息

In [11]:
#基本信息
df=cb_df.loc[:,(
    '代码','转债名称','现价','涨跌',
    '正股名称','股价','涨跌 ',    
    '转股价','转股价值','溢价率',
    '到期时间','剩余(年)',        
    '税前收益','税后收益',           
    '规模(亿)','成交额(万)',
    )]
#按照现价筛选（现价小于100）并排序
df['现价']=df['现价'].astype('float')
df=df[df['现价']<100]
df=df.sort_values('现价',ascending=True)
#重置索引
df=df.reset_index()
del df['index']
df

NameError: name 'cb_df' is not defined

### 1.2.回售、强赎

In [0]:
#回售、强赎信息
df=cb_df.loc[:,(
    '代码','转债名称','现价','转股价',
    '转股起始日','强赎触发价','强赎触发比',        
    '回售起始日','回售触发价','回售触发比','回售价','到期赎回价',
    '转债比','折算率',       
    )]
#按照现价筛选（现价小于100）并排序
df['现价']=df['现价'].astype('float')
df=df[df['现价']<100]
df=df.sort_values('现价',ascending=True)
#重置索引
df=df.reset_index()
del df['index']
df

### 1.3.条款信息

#### 1.3.1.单条查看

In [0]:
#设置要查看的转债代码
cb_id='128016'
df=cb_df[cb_df['代码']==cb_id]
#重置索引
pd.set_option('max_colwidth', 250)
df.index=df['代码']+' '+df['转债名称']
df.index.name=None
df=df.loc[:,(
    '回售条款', '强赎条款','利息','向下修正',    
    )]
df.T

#### 1.3.2.查看所有

In [0]:
pd.set_option('max_colwidth', 150)
cb_df.index=cb_df['代码']+cb_df['转债名称']
df.index.name=None
df=cb_df.loc[:,(
    '回售条款', '强赎条款','向下修正','利息',  
    )]
df

## 2.待发、待上市转债

### 2.1.待申购

In [0]:
#筛选待申购    
df=cbnew_df[cbnew_df['ap_flag']=='B']
df=df.loc[:,(
    '正股代码','正股名称',
    '发行规模(亿)','进展','进展日期',
    '百元股票含权(元)','转股价','现价','涨幅','现价比转股价',    
    )] 
#更改列名
df=df.rename(columns={'进展日期':'申购日期','进展':'申购代码'}) 
df['申购代码']=df['申购代码'].str[5:11]
#重置索引
df=df.reset_index()
del df['index']
df

### 2.2.待上市

In [0]:
#筛选待上市   
df=cbnew_df[cbnew_df['ap_flag']=='E']
df=df.loc[:,(
    '代码','转债名称',
    '正股名称',
    '进展日期',
    '转股价','现价','涨幅',
    '股东配售率','网上规模(亿)',
    '中签率','单账户中签(顶格)','申购户数(万)', 
    '发行规模(亿)',
    )]
df=df.rename(columns={'进展日期':'上市日期',}) 
#重置索引
df=df.reset_index()
del df['index']
df

### 2.3.已申购

In [0]:
#筛选待申购    
df=cbnew_df[cbnew_df['ap_flag']=='C']
df=df.loc[:,(
    '代码','转债名称',
    '正股名称',
    '进展日期',
    '转股价','现价','涨幅',
    '股东配售率','网上规模(亿)',
    '中签率','单账户中签(顶格)','申购户数(万)', 
    '发行规模(亿)',
    )]
df=df.rename(columns={'进展日期':'申购日期',}) 
#重置索引
df=df.reset_index()
del df['index']
df

### 2.4.待发售

In [0]:
#所有待发售
df=cbnew_df[cbnew_df['ap_flag']=='N']
df=df.loc[:,(
    '正股代码','正股名称',
    '发行规模(亿)','进展','进展日期',
    '百元股票含权(元)','转股价','现价','涨幅','现价比转股价',    
    )] 
#重置索引
df=df.reset_index()
del df['index']
df